In [3]:
import pandas as pd
import tkinter as tk
from tkinter import ttk
from tkinter import StringVar
import requests
from io import StringIO
from unidecode import unidecode

df = None
tree = None
colunas_selecionadas = []
filter_comboboxes = {}  # Defina o dicionário filter_comboboxes globalmente

def determinar_delimitador(content):
    delimiters = [',', ';', '\t']
    for delimiter in delimiters:
        if delimiter in content:
            return delimiter
    return ','

def remover_acentos(text):
    return unidecode(text)

def texto_em_maiusculas(text):
    return text.upper()

def carregar_arquivo():
    global df, tree, colunas_selecionadas, filter_comboboxes
    url = entrada_url.get()

    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Erro ao carregar o arquivo da web. Código de status: {response.status_code}")
        return

    content = response.text

    delimitador = determinar_delimitador(content)

    try:
        df = pd.read_csv(StringIO(content), sep=delimitador, on_bad_lines='skip', dtype=str)
    except Exception as e:
        print(f"Erro ao carregar o arquivo: {str(e)}")
        return

    df.columns = df.columns.map(remover_acentos)
    
    colunas = df.columns.tolist()

    if not colunas_selecionadas:
        colunas_selecionadas = colunas[:4]
    else:
        colunas_selecionadas = [col.strip() for col in colunas_selecionadas]
        
    df = df.applymap(texto_em_maiusculas)
    
    for col in colunas_selecionadas:
        filtro = StringVar()
        filtro.set("Todas")
        combobox = ttk.Combobox(frame_filtros, textvariable=filtro)
        combobox['values'] = ["Todas"] + list(df[col].unique())
        combobox.grid(row=0, column=colunas_selecionadas.index(col), sticky="w")
        filter_comboboxes[col] = filtro

    botao_filtrar = tk.Button(frame_filtros, text="Filtrar", command=filtar_tabela)
    botao_filtrar.grid(row=1, column=0, columnspan=len(colunas_selecionadas), pady=5)

    columns = colunas_selecionadas

    tree_frame = ttk.Frame(root)
    tree_frame.grid(row=3, column=0, columnspan=2, padx=10, pady=10, sticky="nsew")
    tree_frame.grid_rowconfigure(0, weight=1)
    tree_frame.grid_columnconfigure(0, weight=1)

    canvas = tk.Canvas(tree_frame)
    canvas.grid(row=0, column=0, sticky="nsew")

    vsb = ttk.Scrollbar(tree_frame, orient="vertical", command=canvas.yview)
    vsb.grid(row=0, column=1, sticky="ns")
    canvas.configure(yscrollcommand=vsb.set)

    tree = ttk.Treeview(canvas, columns=columns, show="headings")
    tree.grid(row=0, column=0, sticky="nsew")

    hsb = ttk.Scrollbar(tree_frame, orient="horizontal", command=canvas.xview)
    hsb.grid(row=1, column=0, sticky="ew")
    canvas.configure(xscrollcommand=hsb.set)

    canvas.create_window((0, 0), window=tree, anchor="nw")
    canvas.update_idletasks()
    canvas.configure(scrollregion=canvas.bbox("all"))

    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, width=100)

    mostrar_tabela(tree)

def mostrar_tabela(tree):
    if df is not None:
        for i in tree.get_children():
            tree.delete(i)
        for index, row in df.iterrows():
            values = [row[col] for col in df.columns]
            tree.insert("", "end", values=values)

def filtar_tabela():
    if df is not None:
        selected_filters = {col: filtro.get() for col, filtro in filter_comboboxes.items()}
        mask = pd.Series([True] * len(df))
        for col, value in selected_filters.items():
            if value != "Todas" and col in df.columns:
                mask &= (df[col] == value)
        for i in tree.get_children():
            tree.delete(i)
        for index, row in df[mask].iterrows():
            values = [row[col] for col in df.columns]
            tree.insert("", "end", values=values)

root = tk.Tk()
root.title("Leitor de CSV Online")

frame_url = tk.Frame(root)
frame_url.grid(row=0, column=0, padx=10, pady=5, sticky="w")
label_url = tk.Label(frame_url, text="URL do CSV:")
label_url.grid(row=0, column=0)
entrada_url = tk.Entry(frame_url)
entrada_url.grid(row=0, column=1)

frame_colunas = tk.Frame(root)
frame_colunas.grid(row=1, column=0, padx=10, pady=5, sticky="w")
label_colunas = tk.Label(frame_colunas, text="Colunas (separadas por vírgula):")
label_colunas.grid(row=0, column=0)
entrada_colunas = tk.Entry(frame_colunas)
entrada_colunas.grid(row=0, column=1)

frame_filtros = tk.Frame(root)
frame_filtros.grid(row=2, column=0, columnspan=2, padx=10, pady=5, sticky="w")

botao_carregar = tk.Button(root, text="Carregar Arquivo", command=carregar_arquivo)
botao_carregar.grid(row=3, column=0, columnspan=2, pady=10)

root.mainloop()